<a href="https://colab.research.google.com/github/LorenzEh/LLM-Vaccine-Hesitency/blob/main/LLM_Data_Imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the libraries

In [ ]:
!pip install -q scikit-learn --upgrade
!pip install -q missingpy
!pip install -q gdown


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.8 MB/s eta 0:00:00


# Import & prepare data

In [ ]:
import pandas as pd
import re
import gdown
import numpy as np
from collections import defaultdict
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt

In [ ]:
gdown.download(id="1iSnyo6sLT6qpv33jwpcLoWV1CqgkjBzC", output="agg_data.csv", quiet=False)
data = pd.read_csv("agg_data.csv")

data.head(10)

Downloading...
From: https://drive.google.com/uc?id=1iSnyo6sLT6qpv33jwpcLoWV1CqgkjBzC
To: /content/agg_data.csv
100%|██████████| 42.7M/42.7M [00:00<00:00, 110MB/s]


,GENDER,AGE_CAT,EDU_CAT,BULA,Gesundheitliche Gefahr: Für mich persönlich,Gesundheitliche Gefahr: Für die österreichische Bevölkerung,Wirtschaftliche Gefahr: Für mich persönlich,Wirtschaftliche Gefahr: Für die österreichische Bevölkerung,Anzahl der Infizierten (eigene Einschätzung),Maßnahmen der Regierung: Angemessenheit,...,"Ukraine-Hilfspaket: BIP schrumpft stark, ALQ steigt stark, EU-Beitrag wenig","Ukraine-Hilfspaket: BIP schrumpft leicht, ALQ steigt stark, EU-Beitrag wenig","Ukraine-Hilfspaket: BIP schrumpft stark, ALQ steigt leicht, EU-Beitrag wenig","Ukraine-Hilfspaket: BIP schrumpft leicht, ALQ steigt leicht, EU-Beitrag wenig",Verantwortlich für Ukraine-Krieg: Russland,Verantwortlich für Ukraine-Krieg: Ukraine,Verantwortlich für Ukraine-Krieg: EU,Verantwortlich für Ukraine-Krieg: NATO,Verantwortlich für Ukraine-Krieg: USA,Links-Rechts-Einstufung: MFG
0,Weiblich,30-39 Jahre,Niedrig (Kat. 1-3),Salzburg,3 = mittelmaessig,3 = mittelmaessig,3 = mittelmaessig,3 = mittelmaessig,10346.153846,4 = eher zu stark,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Weiblich,14-29 Jahre,Niedrig (Kat. 1-3),Oberoesterreich,5 = sehr klein,1 = sehr gross,3 = mittelmaessig,1 = sehr gross,232962.666667,5 = zu extrem,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Maennlich,14-29 Jahre,Hoch (Kat. 5-10),Oberoesterreich,5 = sehr klein,3 = mittelmaessig,3 = mittelmaessig,2 = gross,825.000000,3 = angemessen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Maennlich,65+ Jahre,Hoch (Kat. 5-10),Vorarlberg,4 = klein,3 = mittelmaessig,4 = klein,2 = gross,48611.111111,4 = eher zu stark,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Weiblich,50-64 Jahre,Niedrig (Kat. 1-3),Oberoesterreich,4 = klein,2 = gross,4 = klein,2 = gross,331264.285714,3 = angemessen,...,2,2,3,2,Voll und ganz verantwortlich,"Teils, teils",Eher nicht verantwortlich,Eher verantwortlich,Eher verantwortlich,5
5,Weiblich,30-39 Jahre,Mittel (Kat. 4),Oberoesterreich,4 = klein,2 = gross,3 = mittelmaessig,2 = gross,NaN,3 = angemessen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Maennlich,30-39 Jahre,Hoch (Kat. 5-10),Wien,3 = mittelmaessig,3 = mittelmaessig,2 = gross,2 = gross,102500.000000,3 = angemessen,...,5,6,4,4,"Teils, teils",Eher verantwortlich,Eher verantwortlich,Voll und ganz verantwortlich,Voll und ganz verantwortlich,4
7,Maennlich,40-49 Jahre,Mittel (Kat. 4),Niederoesterreich,3 = mittelmaessig,3 = mittelmaessig,2 = gross,2 = gross,64464.705882,3 = angemessen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Weiblich,30-39 Jahre,Hoch (Kat. 5-10),Steiermark,2 = gross,1 = sehr gross,3 = mittelmaessig,1 = sehr gross,25000.000000,3 = angemessen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Maennlich,40-49 Jahre,Mittel (Kat. 4),Niederoesterreich,3 = mittelmaessig,3 = mittelmaessig,2 = gross,1 = sehr gross,30925.000000,4 = eher zu stark,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def show_stats(df): # function to show number of rows, cols and missing values
  print("missing values: ",df.isna().sum().sum() / (df.shape[0] * df.shape[1])) # missing values
  print("rows: ",df.shape[0]) # nr rows
  print("columns: ", df.shape[1]) # nr columns

show_stats(data) # 43% missing data

missing values:  0.42959645521057094
rows:  3485
columns:  1086


In [ ]:
impfen_missing = (
    data.isna().mean()
    .loc[lambda x: x.index.str.contains('impfen', case=False, regex=False)]
    .mul(100)
    .sort_values()
)
print(impfen_missing) # we need "impfen ehestmöglich" > elemental for vaccine hesitancy


Impfen: Impfpflicht für alle                                                      10.014347
Impfen: Gratis Impfstoff                                                          10.014347
Impfen: Impfpflicht für Berufsgruppen mit hohem Risiko                            17.704448
Impfen: Meisten Personen in meinem Umfeld sind bereit sich impfen zu lassen       19.598278
Aussage: Lasse mich impfen, wenn andere sich zuerst impfen lassen                 22.525108
Impfen: Ehestmöglich                                                              23.414634
Wissen: Bill Gates will Menschheit zwangsimpfen, um viel Geld zu verdienen        25.767575
Impfen: Es ist unfair, dass bestimmte Gruppen die Impfung früher erhalten         35.380201
Meinung anderer Österreicher: Alle müssen sich impfen lassen                      36.814921
Verhalten anderer Österreicher: Sie haben sich impfen lassen                      36.814921
Impfen: Sollte Möglichkeit haben, mich sofort impfen zu lassen                  

In [ ]:
# check the numbers of categories and how many variables there are per category

def check_columns_per_category(data):
  # group columns by topic (e.g., "health", "vaccination")
  topic_columns = defaultdict(list)
  for col in data.columns:
      topic = col.split(":")[0].strip()  # split at ":" and take the first part
      topic_columns[topic].append(col)

  # count variables per topic
  topic_counts = {topic: len(cols) for topic, cols in topic_columns.items()}
  print("Number of variables per topic:")
  for topic, count in topic_counts.items():
      print(f"{topic}: {count}")
  return topic_columns


check_columns_per_category(data)

Number of variables per topic:
GENDER: 1
AGE_CAT: 1
EDU_CAT: 1
BULA: 1
Gesundheitliche Gefahr: 2
Wirtschaftliche Gefahr: 2
Anzahl der Infizierten (eigene Einschätzung): 1
Maßnahmen der Regierung: 2
Geschätzte Dauer der Krise: 2
Auswirkungen: 4
Depressivität: 9
Zuhause verlassen: 9
Wie viele Österreicher denken: 3
Chance: 3
Vertrauen: 15
Überwachung: 13
Demokratiezufriedenheit: 1
Politische Entscheidungen: 6
Aussage: 124
Zufriedenheit Bundesregierung: 5
Auszahlung Unterstützung: 2
Haushaltsgegenstände: 3
Mediennutzung: 21
Soziale Medien: 6
Soziale Medien Aktivität: 3
Risikobereitschaft: 1
Psychologie: 4
Lebenszufriedenheit: 2
Anzahl der Infizierten (amtlich): 1
Rauchverhalten: 2
Alkoholkonsum: 2
Zeit: 6
Emotion politische Lage: 7
Informationen an Staat: 2
EU: 10
Links-Rechts-Selbsteinstufung: 1
Verfügbarkeit: 3
Soziale Kontakte: 1
Wissen: 15
Meinung anderer Österreicher: 8
Verhalten anderer Österreicher: 6
Eigenes Verhalten: 4
Gemeinschaftsgefühl: 3
Nach der Krise: 7
Demokratie: 11
Fina

defaultdict(list,
            {'GENDER': ['GENDER'],
             'AGE_CAT': ['AGE_CAT'],
             'EDU_CAT': ['EDU_CAT'],
             'BULA': ['BULA'],
             'Gesundheitliche Gefahr': ['Gesundheitliche Gefahr: Für mich persönlich',
              'Gesundheitliche Gefahr: Für die österreichische Bevölkerung'],
             'Wirtschaftliche Gefahr': ['Wirtschaftliche Gefahr: Für mich persönlich',
              'Wirtschaftliche Gefahr: Für die österreichische Bevölkerung '],
             'Anzahl der Infizierten (eigene Einschätzung)': ['Anzahl der Infizierten (eigene Einschätzung)'],
             'Maßnahmen der Regierung': ['Maßnahmen der Regierung: Angemessenheit',
              'Maßnahmen der Regierung: Effektivität'],
             'Geschätzte Dauer der Krise': ['Geschätzte Dauer der Krise',
              'Geschätzte Dauer der Krise: Mehr als 6 Monate'],
             'Auswirkungen': ['Auswirkungen: Vergesse Wochentag',
              'Auswirkungen: Fehlt Kontakt',
           

# Delete rows & columns with large amounts of missing data

In [ ]:
data = data.loc[:, data.isnull().mean() <= 0.25] # exclude all columns (variables) which have more then 25% missings
data = data.loc[data.isnull().mean(axis=1) <= 0.25] # exclude all rows (persons) which have more then 25% missings

topics = check_columns_per_category(data) # check the number of columns per category

Number of variables per topic:
GENDER: 1
AGE_CAT: 1
EDU_CAT: 1
BULA: 1
Gesundheitliche Gefahr: 2
Wirtschaftliche Gefahr: 2
Anzahl der Infizierten (eigene Einschätzung): 1
Maßnahmen der Regierung: 2
Geschätzte Dauer der Krise: 2
Auswirkungen: 4
Depressivität: 9
Zuhause verlassen: 9
Chance: 3
Vertrauen: 11
Überwachung: 6
Demokratiezufriedenheit: 1
Politische Entscheidungen: 6
Aussage: 36
Zufriedenheit Bundesregierung: 4
Auszahlung Unterstützung: 1
Haushaltsgegenstände: 3
Risikobereitschaft: 1
Psychologie: 4
Lebenszufriedenheit: 1
Anzahl der Infizierten (amtlich): 1
Zeit: 6
EU: 8
Links-Rechts-Selbsteinstufung: 1
Soziale Kontakte: 1
Wissen: 3
Meinung anderer Österreicher: 6
Verhalten anderer Österreicher: 4
Eigenes Verhalten: 4
Gemeinschaftsgefühl: 3
Demokratie: 9
Finanzielles Auskommen: 1
Wirtschaftslage: 1
Politikinteresse: 1
Verbundenheit: 4
Glück vs. Leistung: 1
News Avoidance: 2
Maßnahme: 12
Schlafqualität: 1
Zufriedenheit: 4
Zukunftserwartung: 3
Populismus: 6
Guter/schlechter Zeitpun

In [ ]:
data.shape # we have 2727 Persons and 231 columns in total

(2727, 231)

Additionally to dropping all columns and rows which have more than 25% missing values, we will keep only those participants (rows) who have answered more than 25% of questions (colums) in each topic.

In [ ]:
# initialize a mask to filter participants
mask = pd.Series(True, index=data.index)

# check each topic for missingness thresholds
for topic, cols in topics.items():
    total_questions = len(cols)
    answered = data[cols].notnull().sum(axis=1)  # count non-NaN answers per participant
    ratio = answered / total_questions  # fraction of questions answered in the topic

    # update mask: keep participants who answered ≥25% of questions in this topic
    mask &= (ratio >= 0.25)

# apply the final mask to filter participants (rows)
data = data[mask]

In [ ]:
data.shape

(1729, 231)

In [ ]:
data.head(10)

,GENDER,AGE_CAT,EDU_CAT,BULA,Gesundheitliche Gefahr: Für mich persönlich,Gesundheitliche Gefahr: Für die österreichische Bevölkerung,Wirtschaftliche Gefahr: Für mich persönlich,Wirtschaftliche Gefahr: Für die österreichische Bevölkerung,Anzahl der Infizierten (eigene Einschätzung),Maßnahmen der Regierung: Angemessenheit,...,"Aussage: Lasse mich impfen, wenn andere sich zuerst impfen lassen",Aussage: Mit einer Impfung kann ich mich selbst schützen,Aussage: Behördlich zugelassene Impfungen sind sicher,Aussage: Behörden informieren hinreichend über Wirkungsweise der Impfstoffe,"Aussage: Mein Alltag ist zu stressig, ich habe keine Zeit für eine Impfung",Corona-Tests: Häufigkeit in den letzten vier Wochen,Impfen: Impfpflicht für Berufsgruppen mit hohem Risiko,Aussage: Alles in allem empfinde ich die Corona-Maßnahmen als gerecht,Ansteckungsgefahr: Treffen von Freunden Zuhause,Impfen: Meisten Personen in meinem Umfeld sind bereit sich impfen zu lassen
3,Maennlich,65+ Jahre,Hoch (Kat. 5-10),Vorarlberg,4 = klein,3 = mittelmaessig,4 = klein,2 = gross,48611.111111,4 = eher zu stark,...,5 = Trifft gar nicht zu,3 = Teils-teils,2 = Trifft eher zu,3 = Teils-teils,5 = Trifft gar nicht zu,Ich habe mich nicht testen lassen.,2 = Trifft eher zu,3 = Teils-teils,3 = mittelmaessig,2 = Trifft eher zu
4,Weiblich,50-64 Jahre,Niedrig (Kat. 1-3),Oberoesterreich,4 = klein,2 = gross,4 = klein,2 = gross,331264.285714,3 = angemessen,...,4 = Trifft eher nicht zu,2 = Trifft eher zu,2 = Trifft eher zu,2 = Trifft eher zu,5 = Trifft gar nicht zu,Mehr als 4 Mal,1 = Trifft voll und ganz zu,2 = Trifft eher zu,4 = klein,1 = Trifft voll und ganz zu
7,Maennlich,40-49 Jahre,Mittel (Kat. 4),Niederoesterreich,3 = mittelmaessig,3 = mittelmaessig,2 = gross,2 = gross,64464.705882,3 = angemessen,...,1 = Trifft voll und ganz zu,2 = Trifft eher zu,3 = Teils-teils,4 = Trifft eher nicht zu,4 = Trifft eher nicht zu,4 Mal,1 = Trifft voll und ganz zu,2 = Trifft eher zu,3 = mittelmaessig,1 = Trifft voll und ganz zu
9,Maennlich,40-49 Jahre,Mittel (Kat. 4),Niederoesterreich,3 = mittelmaessig,3 = mittelmaessig,2 = gross,1 = sehr gross,30925.000000,4 = eher zu stark,...,3 = Teils-teils,2 = Trifft eher zu,3 = Teils-teils,4 = Trifft eher nicht zu,5 = Trifft gar nicht zu,Mehr als 4 Mal,4 = Trifft eher nicht zu,3 = Teils-teils,3 = mittelmaessig,2 = Trifft eher zu
15,Weiblich,65+ Jahre,Mittel (Kat. 4),Niederoesterreich,5 = sehr klein,4 = klein,3 = mittelmaessig,3 = mittelmaessig,41327.586207,5 = zu extrem,...,5 = Trifft gar nicht zu,2 = Trifft eher zu,3 = Teils-teils,4 = Trifft eher nicht zu,5 = Trifft gar nicht zu,Ich habe mich nicht testen lassen.,2 = Trifft eher zu,4 = Trifft eher nicht zu,4 = klein,2 = Trifft eher zu
17,Weiblich,50-64 Jahre,Mittel (Kat. 4),Niederoesterreich,4 = klein,2 = gross,5 = sehr klein,2 = gross,102177.272727,3 = angemessen,...,5 = Trifft gar nicht zu,1 = Trifft voll und ganz zu,1 = Trifft voll und ganz zu,2 = Trifft eher zu,5 = Trifft gar nicht zu,2 Mal,1 = Trifft voll und ganz zu,1 = Trifft voll und ganz zu,1 = sehr gross,2 = Trifft eher zu
20,Weiblich,50-64 Jahre,Mittel (Kat. 4),Oberoesterreich,5 = sehr klein,4 = klein,3 = mittelmaessig,2 = gross,2427.129032,3 = angemessen,...,2 = Trifft eher zu,3 = Teils-teils,3 = Teils-teils,3 = Teils-teils,5 = Trifft gar nicht zu,Ich habe mich nicht testen lassen.,5 = Trifft gar nicht zu,3 = Teils-teils,4 = klein,2 = Trifft eher zu
23,Weiblich,50-64 Jahre,Niedrig (Kat. 1-3),Salzburg,2 = gross,2 = gross,2 = gross,2 = gross,8394.615385,3 = angemessen,...,5 = Trifft gar nicht zu,1 = Trifft voll und ganz zu,1 = Trifft voll und ganz zu,1 = Trifft voll und ganz zu,5 = Trifft gar nicht zu,Ich habe mich nicht testen lassen.,1 = Trifft voll und ganz zu,1 = Trifft voll und ganz zu,4 = klein,1 = Trifft voll und ganz zu
26,Maennlich,65+ Jahre,Mittel (Kat. 4),Oberoesterreich,5 = sehr klein,3 = mittelmaessig,5 = sehr klein,3 = mittelmaessig,5690.925926,3 = angemessen,...,5 = Trifft gar nicht zu,1 = Trifft voll und ganz zu,1 = Trifft voll un

In [ ]:
# identify ordinal columns (with " = " labels, eg. 4 = klein, 3 = mittelmaessig etc...)
ordinal_cols = [
    col for col in data.columns
    if data[col].dropna().astype(str).str.contains(r"\d+ = ").any()
]

# store original categorical metadata BEFORE encoding
category_metadata = {}

# create encoding df
df_encoded = data.copy()

# process ordinal columns first
for col in ordinal_cols:
    df_encoded[col] = df_encoded[col].str.extract(r"(\d+)").astype(float)

# identify nominal columns from ORIGINAL data
nominal_cols = [
    col for col in data.columns
    if (col not in ordinal_cols) and
       (not pd.api.types.is_numeric_dtype(data[col]))
]

# preserve category information
for col in nominal_cols:
    # Convert to categorical using original data's values
    df_encoded[col] = pd.Categorical(data[col])
    category_metadata[col] = {
        'categories': df_encoded[col].cat.categories,
        'ordered': df_encoded[col].cat.ordered
    }

    # Now convert to codes
    df_encoded[col] = df_encoded[col].cat.codes

In [ ]:
df_encoded.head(10)

,GENDER,AGE_CAT,EDU_CAT,BULA,Gesundheitliche Gefahr: Für mich persönlich,Gesundheitliche Gefahr: Für die österreichische Bevölkerung,Wirtschaftliche Gefahr: Für mich persönlich,Wirtschaftliche Gefahr: Für die österreichische Bevölkerung,Anzahl der Infizierten (eigene Einschätzung),Maßnahmen der Regierung: Angemessenheit,...,"Aussage: Lasse mich impfen, wenn andere sich zuerst impfen lassen",Aussage: Mit einer Impfung kann ich mich selbst schützen,Aussage: Behördlich zugelassene Impfungen sind sicher,Aussage: Behörden informieren hinreichend über Wirkungsweise der Impfstoffe,"Aussage: Mein Alltag ist zu stressig, ich habe keine Zeit für eine Impfung",Corona-Tests: Häufigkeit in den letzten vier Wochen,Impfen: Impfpflicht für Berufsgruppen mit hohem Risiko,Aussage: Alles in allem empfinde ich die Corona-Maßnahmen als gerecht,Ansteckungsgefahr: Treffen von Freunden Zuhause,Impfen: Meisten Personen in meinem Umfeld sind bereit sich impfen zu lassen
3,1,4,0,7,4.0,3.0,4.0,2.0,48611.111111,4.0,...,5.0,3.0,2.0,3.0,5.0,4,2.0,3.0,3.0,2.0
4,2,3,2,3,4.0,2.0,4.0,2.0,331264.285714,3.0,...,4.0,2.0,2.0,2.0,5.0,5,1.0,2.0,4.0,1.0
7,1,2,1,2,3.0,3.0,2.0,2.0,64464.705882,3.0,...,1.0,2.0,3.0,4.0,4.0,3,1.0,2.0,3.0,1.0
9,1,2,1,2,3.0,3.0,2.0,1.0,30925.000000,4.0,...,3.0,2.0,3.0,4.0,5.0,5,4.0,3.0,3.0,2.0
15,2,4,1,2,5.0,4.0,3.0,3.0,41327.586207,5.0,...,5.0,2.0,3.0,4.0,5.0,4,2.0,4.0,4.0,2.0
17,2,3,1,2,4.0,2.0,5.0,2.0,102177.272727,3.0,...,5.0,1.0,1.0,2.0,5.0,1,1.0,1.0,1.0,2.0
20,2,3,1,3,5.0,4.0,3.0,2.0,2427.129032,3.0,...,2.0,3.0,3.0,3.0,5.0,4,5.0,3.0,4.0,2.0
23,2,3,2,4,2.0,2.0,2.0,2.0,8394.615385,3.0,...,5.0,1.0,1.0,1.0,5.0,4,1.0,1.0,4.0,1.0
26,1,4,1,3,5.0,3.0,5.0,3.0,5690.925926,3.0,...,5.0,1.0,1.0,1.0,5.0,4,1.0,3.0,4.0,3.0
27,1,3,0,7,2.0,2.0,2.0,2.0,102650.000000,2.0,...,5.0,1.0,2.0,2.0,5.0,1,1.0,4.0,3.0,2.0


In [ ]:
show_stats(df_encoded) # 9% missings, 1729 rows and 231 columns

missing values:  0.03652738239204405
rows:  1729
columns:  231


In [ ]:
print(category_metadata)

{'GENDER': {'categories': Index(['Keine Angabe/Divers', 'Maennlich', 'Weiblich'], dtype='object'), 'ordered': False}, 'AGE_CAT': {'categories': Index(['14-29 Jahre', '30-39 Jahre', '40-49 Jahre', '50-64 Jahre',
       '65+ Jahre'],
      dtype='object'), 'ordered': False}, 'EDU_CAT': {'categories': Index(['Hoch (Kat. 5-10)', 'Mittel (Kat. 4)', 'Niedrig (Kat. 1-3)',
       'keine Angabe'],
      dtype='object'), 'ordered': False}, 'BULA': {'categories': Index(['Burgenland', 'Kaernten', 'Niederoesterreich', 'Oberoesterreich',
       'Salzburg', 'Steiermark', 'Tirol', 'Vorarlberg', 'Wien'],
      dtype='object'), 'ordered': False}, 'Geschätzte Dauer der Krise': {'categories': Index(['1-2 Monate', '3-4 Monate', '5-6 Monate', 'Mehr als 6 Monate',
       'Weiss nicht', 'Weniger als 1 Monat', 'keine Angabe'],
      dtype='object'), 'ordered': False}, 'Depressivität: Ruhig und gelassen': {'categories': Index(['Weiss nicht', 'an manchen Tagen', 'beinahe jeden Tag', 'keine Angabe',
       'mehrm

# Random Forest Imputation
I choose random forest imputation mainly due to two reasons: first, it's relatively uncomplicated to get reasonably good results for both numerical and categorical data, and secondly, I've used it many times, which makes the implementation convenient for me. The workflow is as follows:

1. **Masking Data for Evaluation:** To evaluate the imputation performance, 5% of the data is randomly masked and set to NaN.  While setting a higher percentage would provide more robust metrics, the current value is used to avoid excessive processing time or potential crashes in the Colab environment.

2. **Imputation using Random Forests:** We used a random forest regressor to impute missing values in numerical variables, and a random forest classifier for categorical variables. Many of the ordinal variables that were treated as numerical should probably have been treated as categorical with classifier imputation (especially those which do not have a lot of categories, which are most of them). Due to computational speed and simplicity, I chose to impute all variables which have a numerical value along with their category (eg. 5 = Trifft gar nicht zu) using regression instead of classification, and later rounding the result for decoding. This approach is highly debatable but was done to keep the duration of the imputation process low. In retrospective I would have done this different.

3. **Performance Evaluation and Data Reconstruction:**

 - We calculate the Root Mean Squared Error (RMSE) for the imputed numerical values and accuracy scores for the imputed categorical values.
 -The artificially masked values (from step 1) are replaced with their original, unmasked values to create the final dataset. The original NaN values are replaced with the values estimated by the random forest imputation.


**Improvements:**
- Set the maximum iterations highger (better convergence, but takes way longer to execute the code)
- Use XGBRegressor & Classifier, to run a somewhat similar imputation by using the GPU (which is much faster).
- Think about which values make sense to impute ("Bundesland" is questionable, since it gives really bad accuracy)

Since imputing the whole dataframe takes way too many hours, I decided to exclude certain columns and only keep the first 1000 rows.

In [ ]:
# list of prefixes to include (varibale categories)
prefixes_to_include = ["AGE_CAT", "EDU_CAT", "BULA", "GENDER", "Aussage", "Impfen", "Depressivität", "Zufriedenheit"]

def filter_columns_by_prefix(input_data, prefixes):
    if isinstance(input_data, pd.DataFrame): # works for dataframes
        filtered_cols = [col for col in input_data.columns
                        if any(col.startswith(p) for p in prefixes)]
        return input_data[filtered_cols]
    elif isinstance(input_data, list):  # works for lists
        return [col for col in input_data
               if any(col.startswith(p) for p in prefixes)]
    else:
        raise ValueError("Input must be a DataFrame or list of column names")


# create the new DataFrame with the filtered columns
df_encoded = filter_columns_by_prefix(df_encoded, prefixes_to_include)
df_encoded = df_encoded.iloc[:1000]

data = filter_columns_by_prefix(data, prefixes_to_include)
data = data.iloc[:1000]

# identify ordinal columns using the filtered data
ordinal_cols = [
    col for col in data.columns
    if data[col].dropna().astype(str).str.contains(r"\d+ = ").any()
]

# reprocess nominal columns based on filtered data
nominal_cols = [
    col for col in data.columns
    if (col not in ordinal_cols) and
       (not pd.api.types.is_numeric_dtype(data[col]))
]

In [ ]:
df_encoded.shape
show_stats(df_encoded) # 3% missing values after exlcuding columns and rows

missing values:  0.032370967741935486
rows:  1000
columns:  62


In [ ]:
df_encoded.isnull().sum()

,0
GENDER,0
AGE_CAT,0
EDU_CAT,0
BULA,0
Depressivität: Ruhig und gelassen,0
...,...
Aussage: Behörden informieren hinreichend über Wirkungsweise der Impfstoffe,45
"Aussage: Mein Alltag ist zu stressig, ich habe keine Zeit für eine Impfung",49
Impfen: Impfpflicht für Berufsgruppen mit hohem Risiko,37
Aussage: Alles in allem empfinde ich die Corona-Maßnahmen als gerecht,36


In [ ]:
from sklearn.compose import ColumnTransformer # tomorrow: execute the code and save it (...)

def evaluate_imputation(df_encoded, nominal_cols, data_original, mask_ratio=0.05):
    results = {}
    X_missing = df_encoded.copy()

    true_values = {}
    masked_indices = {}

    # mask values but keep track of original values, we`re setting 5% nan in each column
    for col in df_encoded.columns:
        non_missing_idx = df_encoded[col].dropna().index
        if len(non_missing_idx) < 5:
            continue

        n_mask = max(1, int(mask_ratio * len(non_missing_idx)))
        np.random.seed(42)
        mask_idx = np.random.choice(non_missing_idx, n_mask, replace=False)

        true_values[col] = X_missing.loc[mask_idx, col].copy()
        X_missing.loc[mask_idx, col] = np.nan
        masked_indices[col] = mask_idx

    numerical_cols = [col for col in df_encoded.columns if col not in nominal_cols]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", IterativeImputer(
                estimator=RandomForestRegressor(n_estimators=100, random_state=42),
                initial_strategy='mean',
                max_iter=60,
                random_state=42
            ), numerical_cols),
            ("cat", IterativeImputer(
                estimator=RandomForestClassifier(n_estimators=100, random_state=42),
                initial_strategy="most_frequent",
                max_iter=60,
                random_state=42
            ), nominal_cols)
        ],
        remainder="passthrough"
    )

    X_imputed_array = preprocessor.fit_transform(X_missing)
    all_cols = numerical_cols + nominal_cols
    X_imputed = pd.DataFrame(X_imputed_array, columns=all_cols, index=X_missing.index)

    # evaluate using the imputed values (before restoration)
    for col in masked_indices:
        mask_idx = masked_indices[col]
        y_true = true_values[col]
        y_pred = X_imputed.loc[mask_idx, col]

        if col in nominal_cols:
            cat_column = pd.Categorical(data_original[col])
            max_code = len(cat_column.categories) - 1
            y_pred_round = y_pred.round().astype(int).clip(0, max_code)
            acc = accuracy_score(y_true, y_pred_round)
            results[col] = {"metric": "accuracy", "value": acc}
        else:
            rmse = sqrt(mean_squared_error(y_true, y_pred))
            results[col] = {"metric": "RMSE", "value": rmse}

    # restore original values for artificially masked positions
    for col in masked_indices:
        X_imputed.loc[masked_indices[col], col] = true_values[col]

    return results, X_imputed

# run the function
results, X_imputed_eval = evaluate_imputation(
    df_encoded=df_encoded,
    nominal_cols=nominal_cols,
    data_original=data  # original DataFrame with category metadata
)


/usr/local/lib/python3.11/dist-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In the section below, we check the imputation results by calculating the RMSE for numerical variables and the accuracy score for categorical variables. Two dataframes were generated to showcase the results:
1. One dataframe with the average scores across the various categories
2. One dataframe with the scores for each variable

This approach was chosen due to the large number of variables in the original dataframe—231 in total. Examining the mean values across variable categories offers a concise and informative overview, eliminating the need to inspect each variable individually. Detailed inspection can still be performed in a subsequent step using the provided code.

In [ ]:
# initialize data structures
category_metrics = defaultdict(lambda: defaultdict(list))
variable_metrics = []
category_min_max = defaultdict(lambda: {'min_vals': [], 'max_vals': []})

for var_name, result in results.items():
    # Find category prefix and store metrics
    category = None
    for prefix in prefixes_to_include:
        if var_name.startswith(prefix):
            category = prefix
            # Add to category metrics
            category_metrics[category][result["metric"]].append(result["value"])
            break


    if var_name in nominal_cols:
        # categorical variable need accurarcy score
        var_type = "Categorical"
        accuracy = result["value"]
        rmse = None
        min_val = None
        max_val = None
    else:
        # numerical variables need min and max values as reference for rsme values
        var_type = "Numerical"
        rmse = result["value"]
        accuracy = None
        min_val = df_encoded[var_name].min()
        max_val = df_encoded[var_name].max()
        if category:
            category_min_max[category]['min_vals'].append(min_val)
            category_min_max[category]['max_vals'].append(max_val)

    variable_metrics.append({
        "Variable": var_name,
        "Category": category,
        "Type": var_type,
        "Accuracy": accuracy,
        "RMSE": rmse,
        "Min": min_val,
        "Max": max_val
    })

# create individual variable metrics dataframe, for each variable
metrics_df = pd.DataFrame(variable_metrics)

# create category averages dataframe
avg_df = pd.DataFrame([
    {
        "Category": cat,
        "Metric": metric,
        "Average Value": round(np.mean(vals), 4),
        "N Variables": len(vals)
    }
    for cat, metrics in category_metrics.items()
    for metric, vals in metrics.items()
]).sort_values(["Category", "Metric"])

# create category min/max dataframe
min_max_df = pd.DataFrame([
    {
        "Category": cat,
        "Min Value": min(vals['min_vals']),
        "Max Value": max(vals['max_vals']),
        "N Variables": len(vals['min_vals'])
    }
    for cat, vals in category_min_max.items()
]).sort_values("Category")

# merge category statistics
rmse_and_accuracy = pd.merge(
    avg_df,
    min_max_df[["Category", "Min Value", "Max Value"]],
    on="Category",
    how="left"
)


In [ ]:
print("\nCategory-level Statistics:")
print(rmse_and_accuracy)


Category-level Statistics:
        Category    Metric  Average Value  N Variables  Min Value  Max Value
0        AGE_CAT  accuracy         0.2600            1        NaN        NaN
1        Aussage      RMSE         0.8645           36        1.0        5.0
2           BULA  accuracy         0.0400            1        NaN        NaN
3  Depressivität  accuracy         0.5178            9        NaN        NaN
4        EDU_CAT  accuracy         0.4000            1        NaN        NaN
5         GENDER  accuracy         0.5400            1        NaN        NaN
6         Impfen      RMSE         0.8816            5        1.0        5.0
7  Zufriedenheit      RMSE         0.6790            4        1.0        5.0
8  Zufriedenheit  accuracy         0.4150            4        1.0        5.0


The accuracy and RMSE scores reveal differences in the effectiveness of the imputation across variables. For instance, BULA (Bundesland) has an accuracy of only 4%, indicating that the model was unable to identify any meaningful patterns in the data to estimate the missing values. However, since BULA does not contain any actual missing values and the artificially masked entries were restored with the original data, this is not a concern for subsequent analysis. The variable Zufriedenheit displays both minimum and maximum accuracy values due to mixed data types within the category. This is merely a visual artifact and does not affect the actual results.

Lastly I decoded the dataframe, which makes it easier for the LLM to understand the input. This process was more complicated than expected

In [ ]:
data_imputed_decoded = X_imputed_eval.copy()

for col in nominal_cols:
    # use preserved metadata from original data
    data_imputed_decoded[col] = pd.Categorical.from_codes(
        codes=data_imputed_decoded[col].round().astype(int),
        categories=category_metadata[col]['categories'],
        ordered=category_metadata[col]['ordered']
    )

    # restore original data type if needed
    if data[col].dtype == 'object':
        data_imputed_decoded[col] = data_imputed_decoded[col].astype('object')


# create the ordinal mapping
ordinal_mappings = {}

for col in ordinal_cols:
    # get all possible values from original data format
    unique_values = data[col].dropna().unique()

    # find min/max to handle numeric ranges with missing labels
    numeric_values = []
    for val in unique_values:
        if isinstance(val, str) and " = " in val:
            num = int(val.split(" = ")[0])
            numeric_values.append(num)

    if numeric_values:
        min_val = min(numeric_values)
        max_val = max(numeric_values)
    else:
        min_val, max_val = 0, 0

    # create comprehensive mapping
    mapping = {}
    for val in unique_values:
        if isinstance(val, str) and " = " in val:
            num, label = val.split(" = ", 1)
            mapping[int(num)] = label.strip()

    # fill in missing numbers with default labels
    for num in range(min_val, max_val + 1):
        if num not in mapping:
            # use number as default label if no explicit mapping
            mapping[num] = str(num)

    ordinal_mappings[col] = mapping


for col in ordinal_cols:
    if col in data_imputed_decoded.columns:
        # convert to integer with pandas nullable type
        imputed_vals = (
            data_imputed_decoded[col]
            .round()
            .astype('Int64') # without this the decoding would not work (tranform the floats form the regression back to integers)
        )

        # map using our comprehensive ordinal mapping
        decoded = imputed_vals.map(ordinal_mappings[col])

        # get ordered categories from original data structure
        all_nums = sorted(ordinal_mappings[col].keys())
        ordered_categories = [ordinal_mappings[col][n] for n in all_nums]

        # convert to ordered categorical
        data_imputed_decoded[col] = pd.Categorical(
            decoded,
            categories=ordered_categories,
            ordered=True
        )

# handle remaining data
for col in ordinal_cols:
    if col in data_imputed_decoded.columns:
        # Use mode of original data's labels
        most_common_label = data[col].value_counts().index[0].split(" = ")[-1]
        data_imputed_decoded[col] = data_imputed_decoded[col].fillna(most_common_label)


In [ ]:
print(ordinal_cols)

['Aussage: Akzeptabel höhere Steuern zu zahlen', 'Aussage: Lebensweise ändern', 'Aussage: Wichtigeres als Corona im Leben', 'Aussage: Behauptungen über Corona übertrieben', 'Aussage: Solange andere Verhalten nicht ändern keine Verhaltensänderung', 'Zufriedenheit Bundesregierung: Bilanz', 'Zufriedenheit Bundesregierung: Bekämpfung der Arbeitslosigkeit', 'Zufriedenheit Bundesregierung: Stützung der kleinen und mittleren Unternehmen', 'Zufriedenheit Bundesregierung: Stützung der Wirtschaft allgemein', 'Aussage: Beitrag zur Finanzierung durch alle Personen gleich', 'Aussage: Kluft zwischen Arm und Reich wird durch Krise größer', 'Aussage: Neue Steuern schaden Wirtschaftsstandort Österreich', 'Aussage: Politik muss soziale Ungleichheit bekämpfen', 'Aussage: Sozialstaat macht Menschen träge und faul', 'Aussage: Arbeitslosigkeit durch Staatsschulden bekämpfen', 'Aussage: Wenig Eingriff in Wirtschaft durch Politik ', 'Aussage: Bevorzugung von Frauen bei gleicher Qualifikation ', 'Aussage: Ausw

In [ ]:
data_imputed_decoded.head(100)

,Aussage: Akzeptabel höhere Steuern zu zahlen,Aussage: Lebensweise ändern,Aussage: Wichtigeres als Corona im Leben,Aussage: Behauptungen über Corona übertrieben,Aussage: Solange andere Verhalten nicht ändern keine Verhaltensänderung,Zufriedenheit Bundesregierung: Bilanz,Zufriedenheit Bundesregierung: Bekämpfung der Arbeitslosigkeit,Zufriedenheit Bundesregierung: Stützung der kleinen und mittleren Unternehmen,Zufriedenheit Bundesregierung: Stützung der Wirtschaft allgemein,Aussage: Beitrag zur Finanzierung durch alle Personen gleich,...,Depressivität: So niedergeschlagen,Depressivität: Glücklich,Depressivität: Sehr nervös,Depressivität: Ängstlich,Depressivität: Bedrückt und traurig,Depressivität: Voller Energie,Zufriedenheit: Familienleben,Zufriedenheit: Partnerschaft,Zufriedenheit: Beruf,Zufriedenheit: Freizeit
3,Teils-teils,Teils-teils,Trifft voll und ganz zu,Teils-teils,Trifft gar nicht zu,Sehr unzufrieden,Eher unzufrieden,Sehr unzufrieden,Teils-teils,Trifft gar nicht zu,...,nie,keine Angabe,nie,nie,nie,mehrmals die Woche,sehr zufrieden,sehr zufrieden,trifft nicht zu (keine Familie/ Partner/in/ ni...,eher zufrieden
4,Trifft eher nicht zu,Trifft eher zu,Teils-teils,Teils-teils,Trifft eher nicht zu,Eher zufrieden,Teils-teils,Eher zufrieden,Eher zufrieden,Teils-teils,...,an manchen Tagen,mehrmals die Woche,an manchen Tagen,nie,an manchen Tagen,an manchen Tagen,sehr zufrieden,sehr zufrieden,trifft nicht zu (keine Familie/ Partner/in/ ni...,eher zufrieden
7,Trifft gar nicht zu,Teils-teils,Trifft eher zu,Teils-teils,Trifft eher nicht zu,Teils-teils,Eher unzufrieden,Teils-teils,Teils-teils,Trifft eher nicht zu,...,nie,beinahe jeden Tag,nie,nie,nie,mehrmals die Woche,eher zufrieden,sehr zufrieden,eher zufrieden,eher zufrieden
9,Trifft eher nicht zu,Teils-teils,Teils-teils,Trifft eher zu,Trifft eher nicht zu,Teils-teils,Sehr unzufrieden,Sehr unzufrieden,Eher unzufrieden,Teils-teils,...,nie,beinahe jeden Tag,nie,nie,nie,mehrmals die Woche,eher zufrieden,eher zufrieden,eher zufrieden,eher zufrieden
15,Trifft gar nicht zu,Teils-teils,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft eher zu,Teils-teils,Sehr zufrieden,Eher zufrieden,Sehr zufrieden,Teils-teils,...,nie,beinahe jeden Tag,nie,nie,nie,an manchen Tagen,eher zufrieden,eher zufrieden,trifft nicht zu (keine Familie/ Partner/in/ ni...,eher zufrieden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Teils-teils,Trifft eher zu,Trifft eher nicht zu,Teils-teils,Trifft gar nicht zu,Eher zufrieden,Teils-teils,Teils-teils,Teils-teils,Teils-teils,...,nie,an manchen Tagen,an manchen Tagen,an manchen Tagen,nie,an manchen Tagen,sehr zufrieden,sehr zufrieden,trifft nicht zu (keine Familie/ Partner/in/ ni...,eher zufrieden
197,Trifft gar nicht zu,Teils-teils,Trifft voll und ganz zu,Trifft eher nicht zu,Trifft eher nicht zu,Eher zufrieden,Eher zufrieden,Eher unzufrieden,Eher zufrieden,Trifft eher nicht zu,...,nie,beinahe jeden Tag,nie,nie,nie,beinahe jeden Tag,eher zufrieden,trifft nicht zu (keine Familie/ Partner/in/ ni...,eher zufrieden,eher zufrieden
198,Trifft eher zu,Trifft eher zu,Trifft voll und ganz zu,Trifft voll und ganz zu,Trifft gar nicht zu,Sehr zufrieden,Eher zufrieden,Eher zufrieden,Eher zufrieden,Trifft eher zu,...,nie,taeglich,nie,nie,nie,taeglich,sehr zufrieden,sehr zufrieden,eher zufrieden,eher zufrieden
200,Teils-teils,Trifft eher zu,Teils-teils,Teils-teils,Teils-teils,Teils-teils,Teils-teils,Eher zufrieden,Eher zufrieden,Trifft eher zu,...,nie,beinahe jeden Tag,an manchen Tagen,nie,an manchen Tagen,an manchen Tagen,sehr zufrieden,eher zufrieden,eher zufrieden,weniger zufrieden


In [ ]:
print(data_imputed_decoded.isna().sum().sum()) # check for missing values

0


In [ ]:
# save data
#from google.colab import drive
#drive.mount("/content/drive")
#save_path = "/content/drive/MyDrive/LLM Vaccine Hesitency/data_imputed_decoded.csv"

# Save the DataFrame
#data_imputed_decoded.to_csv(save_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
